In [53]:
%load_ext autoreload
%autoreload 2

from models import Detr
from utils import load_config
import torch

config = load_config(r"configs\train.yaml")

model = Detr(
    train=True,
    lr=1e-4,
    lr_backbone=1e-5,
    weight_decay=1e-4,
    pretrained='facebook/detr-resnet-101-dc5',
    checkpoint = None,
    config=config)
model.train()

device = torch.device('cuda:0')
model.to(device)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Some weights of the model checkpoint at facebook/detr-resnet-101-dc5 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DetrForObjectDetection were not initialized from the model chec

Detr(
  (model): DetrForObjectDetection(
    (model): DetrModel(
      (backbone): DetrConvModel(
        (conv_encoder): DetrConvEncoder(
          (model): FeatureListNet(
            (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
            (bn1): DetrFrozenBatchNorm2d()
            (act1): ReLU(inplace=True)
            (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
            (layer1): Sequential(
              (0): Bottleneck(
                (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (bn1): DetrFrozenBatchNorm2d()
                (act1): ReLU(inplace=True)
                (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                (bn2): DetrFrozenBatchNorm2d()
                (drop_block): Identity()
                (act2): ReLU(inplace=True)
                (aa): Identity()
                (conv3): Conv2d(64, 256

In [54]:
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import Trainer

# Set up checkpointing and trainer
checkpoint_callback = ModelCheckpoint(
    monitor="validation/loss",
    filename="detr-{epoch:02d}-{validation/loss:.2f}",
    save_top_k=3,
    mode="min",
    save_last=True,
    dirpath=r"checkpoints"
)

trainer = Trainer(
    max_epochs=50,
    log_every_n_steps=10,
    callbacks=[checkpoint_callback],
    val_check_interval=0.5,  # Run validation 2 times per epoch
    devices=1,
    accelerator="gpu",
    gradient_clip_val=0.1,
    accumulate_grad_batches=48
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [55]:
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type                   | Params | Mode 
--------------------------------------------------------------------
0 | model            | DetrForObjectDetection | 60.4 M | train
1 | simplified_tbnet | SimplifiedTbNet        | 263 K  | train
--------------------------------------------------------------------
60.5 M    Trainable params
222 K     Non-trainable params
60.7 M    Total params
242.822   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s][{'size': tensor([800, 984]), 'image_id': tensor([0]), 'class_labels': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), 'boxes': tensor([[0.0288, 0.0215, 0.0535, 0.0228],
        [0.4928, 0.0215, 0.0844, 0.0228],
        [0.7191, 0.0215, 0.0556, 0.0228],
        [0.9671, 0.0215, 0.0576, 0.0228],
        [0.0484, 0.0557, 0.0926, 0.0253],
        [0.9578, 0.0557, 0.0391, 0.0253],
        [0.0319, 0.0911, 0.0309, 0.0253],
        [0.4702, 0.0911, 0.0391, 0.0253],
        [0.0319, 0.1266, 0.0309, 0.0253],
        [0.4702, 0.1266, 0.0391, 0.0253],
        [0.7325, 0.1266, 0.0823, 0.0253],
        [0.0278, 0.1620, 0.0514, 0.0253],
        [0.9578, 0.1620, 0.0391, 0.0253],
        [0.0350, 0.1975, 0.0370, 0.0253],
        [0.4702, 0.1975, 0.0391, 0.0253],
       

c:\Users\remote desktop\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_lightning\utilities\data.py:78: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 1. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


[{'size': tensor([ 330, 1333]), 'image_id': tensor([1]), 'class_labels': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1]), 'boxes': tensor([[0.0987, 0.1441, 0.1891, 0.1525],
        [0.4160, 0.1441, 0.0840, 0.1525],
        [0.5105, 0.1441, 0.0462, 0.1525],
        [0.6324, 0.1441, 0.1050, 0.1525],
        [0.7416, 0.1441, 0.0546, 0.1525],
        [0.8697, 0.1441, 0.0588, 0.1525],
        [0.9601, 0.1441, 0.0630, 0.1525],
        [0.0945, 0.3729, 0.1807, 0.1695],
        [0.4265, 0.3729, 0.0630, 0.1695],
        [0.5189, 0.3729, 0.0630, 0.1695],
        [0.5987, 0.3729, 0.0378, 0.1695],
        [0.7563, 0.3729, 0.0840, 0.1695],
        [0.8529, 0.3729, 0.0252, 0.1695],
        [0.9580, 0.3729, 0.0588, 0.1695],
        [0.1765, 0.6102, 0.3445, 0.1695],
        [0.4265, 0.6102, 0.0630, 0.1695],
        [0.5189, 0.6102, 0.0630, 0.1695],
        [0.5987, 0.6102, 0.0378, 0.1695],
        [0.7647, 0.6102, 0.1008, 0.1695],
        [0.8529, 0

c:\Users\remote desktop\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Epoch 0:   0%|          | 0/20 [00:00<?, ?it/s] [{'size': tensor([ 722, 1333]), 'image_id': tensor([5]), 'class_labels': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), 'boxes': tensor([[0.0916, 0.0809, 0.1753, 0.0588],
        [0.6693, 0.0809, 0.3187, 0.0588],
        [0.1016, 0.2096, 0.1952, 0.0662],
        [0.7092, 0.2096, 0.3984, 0.0662],
        [0.1235, 0.2904, 0.2390, 0.0662],
        [0.7012, 0.2904, 0.3825, 0.0662],
        [0.0637, 0.3750, 0.1195, 0.0588],
        [0.7072, 0.3750, 0.3944, 0.0588],
        [0.0857, 0.4596, 0.1633, 0.0662],
        [0.7092, 0.4596, 0.3984, 0.0662],
        [0.0896, 0.5404, 0.1713, 0.0662],
        [0.6952, 0.5404, 0.3705, 0.0662],
        [0.1096, 0.6250, 0.2112, 0.0588],
        [0.7012, 0.6250, 0.3825, 0.0588],
        [0.0916, 0.7096, 0.1753, 0.0662],
        [0.6813, 0.7096, 0.3426, 0.0662],
        [0.1135, 0.7941, 0.2191, 0.0588],
        [0.6912, 0.7941, 0.3625, 0.0588],
        [0.0936, 0.8750, 0.1793, 0.058

c:\Users\remote desktop\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_lightning\trainer\call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [60]:
import torch
import torch.nn.functional as F
from torch.nn import Linear
from torch_geometric.nn import GCNConv

# Define a simple model class
class SimpleModel(torch.nn.Module):
    def __init__(self, edge_index, num_nodes, d_model, num_classes):
        super(SimpleModel, self).__init__()
        self.edge_index = edge_index
        self.num_nodes = num_nodes

        # Define GCN layers
        self.conv1 = GCNConv(d_model, d_model)
        self.conv2 = GCNConv(d_model, d_model)

        # Define linear layers
        self.lin1 = Linear(d_model * 2, d_model)
        self.lin_final = Linear(d_model, num_classes)

    def forward(self, hidden_state, pred_bboxes):

        batch_size, num_nodes, d_model = hidden_state.shape

        hidden_state_reshaped = hidden_state.view(-1, d_model)

        edge_index_batch = self.edge_index.repeat(1, batch_size)
        batch_offsets = torch.arange(batch_size).repeat_interleave(self.edge_index.size(1)) * self.num_nodes
        edge_index_batch += batch_offsets
        edge_index_batch = edge_index_batch.to(hidden_state_reshaped.device)

        x = self.conv1(hidden_state_reshaped, edge_index_batch)
        x = F.relu(x)
        x = self.conv2(x, edge_index_batch)
        x = F.relu(x)

        x = x.view(batch_size, num_nodes, d_model)

        x1 = x[:, self.edge_index[0]]
        x2 = x[:, self.edge_index[1]]
        xpair = torch.cat((x1, x2), dim=-1)
        xpair = F.relu(self.lin1(xpair))

        xfin = self.lin_final(xpair)
        probs = F.log_softmax(xfin, dim=-1)

        bbox_pairs = torch.cat((
            pred_bboxes[:, self.edge_index[0]],
            pred_bboxes[:, self.edge_index[1]]
        ), dim=-1)

        return probs, bbox_pairs

# let's create a test model with only 2 nodes and 2 batches (4 bboxes in total)
batch_size = 2
num_nodes = 2
d_model = 5
num_classes = 3
edge_index = torch.tensor([[0, 1], [1, 0]])
model = SimpleModel(edge_index=edge_index, num_nodes=num_nodes, d_model=d_model, num_classes=num_classes)
hidden_state = torch.randn(batch_size, num_nodes, d_model)

# here we define a raw gt for testing
# in this case, it's a table with two horizontally adjacent cells
gt_html = ["<thead>","<td>","</td>","<td>","</td>","</thead>"]

# gt bboxes in the (xmin, ymin, width, height) COCO format
#   0 1 2 3 4 5 6 7 8 9
# 0 +-----+
# 1 |  1  |     +-----+
# 2 +-----+     |  2  |
# 3             |     |
# 4   + - +     +-----+
# 5   | 3 |
# 6   + - + 
# 7     +-----------+
# 8     |     4     |
# 9     +-----------+
gt_bboxes = [[0,0,3,2],[6,1,3,3],[1,4,2,2],[2,7,6,2]]
pred_bboxes = torch.tensor([[[0,0,3,2],[6,1,3,3]],[[1,4,2,2],[2,7,6,2]]])

# now let's run the model to get the predicted bbox pairs
probs, bbox_pairs = model(hidden_state, pred_bboxes)

# tensor([[[0, 0, 3, 2, 6, 1, 3, 3],
#          [6, 1, 3, 3, 0, 0, 3, 2]],

#         [[1, 4, 2, 2, 2, 7, 6, 2],
#          [2, 7, 6, 2, 1, 4, 2, 2]]])

# now we can compare bbox pairs against gt bboxes based on some criterion
for batch in bbox_pairs:
    for edge in batch:
        # let's split the edge data into bbox 1 and bbox 2
        # we pause here for now because we need to define the gt data structure before proceeding

tensor([[[0, 0, 3, 2, 6, 1, 3, 3],
         [6, 1, 3, 3, 0, 0, 3, 2]],

        [[1, 4, 2, 2, 2, 7, 6, 2],
         [2, 7, 6, 2, 1, 4, 2, 2]]])
